<a href="https://colab.research.google.com/github/podo47/DM_Classification/blob/main/DM_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [211]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [212]:
import warnings
warnings.filterwarnings('ignore')

## Import data

In [213]:
train_path = "/content/drive/MyDrive/Data_Mining/Classification/training.txt"
test_path = "/content/drive/MyDrive/Data_Mining/Classification/test.txt"

def read_file(path):
  trans = []
  with open(path) as ip:
    for line in ip:
        sp = line.strip('\n')
        sp = sp.strip('{}')
        sp = sp.split(',')
        trans.append(sp)
  return trans

train = read_file(train_path)
test = read_file(test_path)

## Data preprocessing

### Modify the format of input and read to list

In [214]:
def modify(data):
  data_list = []
  for row in data:
    row_list = []
    f_dict = {0:np.nan,1:np.nan,2:np.nan,3:np.nan,4:np.nan}
    for item in row:
      id = int(item[0])
      f_dict[id] = item[2:]
    
    for i in range(len(f_dict)):
      row_list.append(f_dict[i])

    data_list.append(row_list)
  
  return data_list

In [215]:
train = modify(train)
test = modify(test)

### Read the data from list to dataframe

In [216]:
train_df = pd.DataFrame(train)
train_df.columns = ['status','children','member','age','income']
test_df = pd.DataFrame(test)
test_df.columns = ['status','children','member','age','income']

### Dealing the missing values 

In [217]:
train_df['status'] = train_df['status'].fillna('S')
test_df['status'] = test_df['status'].fillna('S')
train_df['member'] = train_df['member'].fillna('Basic')
test_df['member'] = test_df['member'].fillna('Basic')

train_df['children'] = train_df['children'].apply(pd.to_numeric)
train_df['children'] = train_df['children'].fillna(value=train_df['children'].mean())
test_df['children'] = test_df['children'].apply(pd.to_numeric)
test_df['children'] = test_df['children'].fillna(value=test_df['children'].mean())
train_df['age'] = train_df['age'].apply(pd.to_numeric)
train_df['age'] = train_df['age'].fillna(value=train_df['age'].mean())
test_df['age'] = test_df['age'].apply(pd.to_numeric)
test_df['age'] = test_df['age'].fillna(value=test_df['age'].mean())
train_df['income'] = train_df['income'].apply(pd.to_numeric)
train_df['income'] = train_df['income'].fillna(value=train_df['income'].mean())
test_df['income'] = test_df['income'].apply(pd.to_numeric)
test_df['income'] = test_df['income'].fillna(value=test_df['income'].mean())

train_df = train_df.round(0)
test_df = test_df.round(0)

### Label Encoding

In [218]:
train_df['status'] = train_df['status'].map({'M': 1, 'S': 0})
train_df['member'] = train_df['member'].map({'Basic': 0, "Normal":1, "Silver":2, "Gold":3})

test_df['status'] = test_df['status'].map({'M': 1, 'S': 0})
test_df['member'] = test_df['member'].map({'Basic': 0, "Normal":1, "Silver":2, "Gold":3})


In [219]:
train_df.head()

,status,children,member,age,income
0,0,1.0,0,39,100000
1,1,2.0,3,85,80000
2,1,1.0,3,90,20000
3,0,2.0,1,31,20000
4,1,2.0,0,49,40000


### Declare feature vector and target variable

In [220]:
train_X = train_df.drop(['member'], axis=1)
train_y = train_df['member']
X_test = test_df.drop(['member'], axis=1)
y_test = test_df['member']

## Naïve Bayer classifier

In [221]:
import math
class Gaussian_Bayes():
  def __init__(self):
    pass
  # Build the model
  def fit(self,X_train,y_train):
    self.training_data = X_train.copy()
    self.gb_y_train = pd.DataFrame(y_train.values, columns = ['claim'])
    self.training_data['claim'] = y_train.values
    self.find_mean = self.training_data.groupby("claim").mean() # Mean for gaussian
    self.find_var = self.training_data.groupby("claim").var() # Variance for gaussian
    self.prior_prob = self.Prior()
    return self
  
  # Prior probability
  def Prior(self):
    label = self.gb_y_train['claim'].value_counts().sort_index()
    pri = []
    for c in label :
      pri.append(c/sum(label))
    prior_prob = np.array(pri)
    return prior_prob
    
  # Calculate with gaussian distribution
  def Distribution(self,mu,variance,feature):
    norm = 1/np.sqrt(2*math.pi*variance)*np.exp(-(feature-mu)**2/(2*variance)) # Normal distribution
    norm = pd.DataFrame(norm).prod(axis=1) # P(x1,x2,...,xn|yi)
    return norm

  # Prediction
  def predict(self,X_test):
    final_prob = []
    for feature in X_test.values :
      prob=self.Distribution(self.find_mean,self.find_var,feature)*self.prior_prob
      final_prob.append(prob)
    result=np.argmax(final_prob, axis=1)  # argmax for prediction
    return result

In [222]:
nb = Gaussian_Bayes()

### Fit the model and predict the class

In [223]:
nb.fit(train_X,train_y)
# Test set prediction
y_test_nb = nb.predict(X_test)
# Train set prediction
y_train_nb = nb.predict(train_X)

### Check accuracy score

In [224]:
from sklearn.metrics import accuracy_score

In [225]:
# Train set accuracy
train_accuracy_nb = accuracy_score(train_y,y_train_nb)
# Test set accuracy
test_accuracy_nb = accuracy_score(y_test,y_test_nb)

print('Test set accuracy : {0:0.4f}'. format(test_accuracy_nb))
# Show the train set accuracy to check if overfitting
print('Train set accuracy : {0:0.4f}'. format(train_accuracy_nb))

Test set accuracy : 0.6540
Train set accuracy : 0.6130


## Output as csv

In [226]:
result_df = pd.DataFrame(test)
result_df.columns = ['status','children','member','age','income']
result_df['member'] = result_df['member'].fillna('Basic')

result_df['predict_member_card'] = y_test_nb
result_df['predict_member_card'] = result_df['predict_member_card'].map({0: 'Basic', 1:"Normal", 2:"Silver", 3:"Gold"})

In [227]:
result_df.head()

,status,children,member,age,income,predict_member_card
0,M,1,Basic,59,60000,Basic
1,NaN,3,Silver,43,160000,Gold
2,M,2,Basic,52,120000,Basic
3,M,2,Normal,30,60000,Basic
4,NaN,NaN,Basic,73,60000,Basic


In [228]:
output_path = "/content/drive/MyDrive/Data_Mining/Classification/output.csv"
result_df.to_csv(output_path,header=True)